# Model II Galaxy Classifier (Spiral, Elliptical, Odd objects)

**Most of this code is made by the authors of the paper of Ghaderi et al. (2025) (https://iopscience.iop.org/article/10.3847/1538-4365/ada8ab) and taken from the GitHub Repository: https://github.com/hmddev1/machine_learning_for_morphological_galaxy_classification**

**This notebook classifies the images of galaxies to spiral and elliptical galaxies and odd objects using the Zernike Moments and the 1D-CNN (Model II).**

**Purpose of this notebook: Run Model II for 10 iterations, and obtain results to reproduce table 4 of Ghaderi et al. (2025).**

In [ ]:
path = '/content/drive/Shared drives/DLP Project/Project/Models/Galaxy Models/Looping each Model'
os.chdir(path)
%run imports.py
%matplotlib inline
import plotting
roc_curves = {}

# Paths to images
spath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_spiral'
epath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_elliptical'
opath = r'/content/drive/Shared drives/DLP Project/Project/Data/galaxy/images/cropped_odd'

# Default image size and zernike order.
image_size = 200
zernike_order = 45

# Loading the ZMs and concatenating to a consolidated dataset
spiral_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/spiral_zms.csv')
elliptical_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/elliptical_zms.csv')
odd_data = pd.read_csv('/content/drive/Shared drives/DLP Project/Project/odd_zms.csv')

spiral_data.drop("Unnamed: 0", axis = 1, inplace = True)
elliptical_data.drop("Unnamed: 0", axis = 1, inplace = True)
odd_data.drop("Unnamed: 0", axis = 1, inplace = True)

all_zm_data = np.concatenate([spiral_data, elliptical_data, odd_data])
np.shape(all_zm_data)

spiral_label = [0] * len(spiral_data)
elliptical_label = [1] * len(elliptical_data)
odd_label = [2] * len(odd_data)

all_labels = np.concatenate([spiral_label, elliptical_label, odd_label])
len(all_labels)

In [ ]:
num_iterations = 10
metrics_list = []  # Store metrics for each iteration

for iteration in range(num_iterations):
    print(f"Iteration {iteration + 1}...")

    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
        all_zm_data, all_labels, np.arange(len(all_labels)),
        test_size=0.25, shuffle=True, random_state=None)

    y_train_encoded = to_categorical(y_train, num_classes=3)

    class_weights = {0: len(all_zm_data) / (3 * len(spiral_data)),
                     1: len(all_zm_data) / (3 * len(elliptical_data)),
                     2: len(all_zm_data) / (3 * len(odd_data))}

    # Building the model
    x = Input(shape=(all_zm_data.shape[1], 1))

    # Hidden layers
    c0 = Conv1D(256, kernel_size=3, strides=2, padding="same")(x)
    b0 = BatchNormalization()(c0)
    m0 = MaxPooling1D(pool_size=2)(b0)
    d0 = Dropout(0.1)(m0)

    c1 = Conv1D(128, kernel_size=3, strides=2, padding="same")(d0)
    b1 = BatchNormalization()(c1)
    m1 = MaxPooling1D(pool_size=2)(b1)
    d1 = Dropout(0.1)(m1)

    c2 = Conv1D(64, kernel_size=3, strides=2, padding="same")(d1)
    b2 = BatchNormalization()(c2)
    m2 = MaxPooling1D(pool_size=2)(b2)
    d2 = Dropout(0.1)(m2)

    f = Flatten()(d2)

    # Output layer
    de0 = Dense(64, activation='relu')(f)
    de1 = Dense(32, activation='relu')(de0)
    de2 = Dense(3, activation='softmax')(de1)

    model = Model(inputs=x, outputs=de2, name="cnn_zm_45_galaxy_nonegalaxy")
    loss = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

    # Training the model
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    b_size = 64
    e_num = 30

    history = model.fit(
        X_train, y_train_encoded,
        batch_size=b_size,
        epochs=e_num,
        class_weight=class_weights,
        verbose=1,
        callbacks=es,
        validation_split=0.1)

    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)


    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred_labels)

    # Performance metrics (per-class)
    recall_per_class = recall_score(y_test, y_pred_labels, average=None)
    precision_per_class = precision_score(y_test, y_pred_labels, average=None)
    f1_per_class = f1_score(y_test, y_pred_labels, average=None)
    accuracy = accuracy_score(y_test, y_pred_labels) # overall

    # Compute per-class TSS
    tss_per_class = {}
    for i, class_name in enumerate(['Spiral', 'Elliptical', 'Odd']):
        tp = cm[i, i]
        fn = np.sum(cm[i, :]) - tp
        fp = np.sum(cm[:, i]) - tp
        tn = np.sum(cm) - (tp + fn + fp)
        tss_per_class[class_name] = (tp / (tp + fn + 1e-6)) - (fp / (fp + tn + 1e-6))

    # Store metrics in a list
    metrics_list.append({
        'Iteration': iteration + 1,
        'Recall per Class': recall_per_class,
        'Precision per Class': precision_per_class,
        'F1 per Class': f1_per_class,
        'Accuracy': accuracy,
        'TSS per Class': tss_per_class
    })


for result in metrics_list:
    print(result)